In [11]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import cv2

from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.utils.class_weight import compute_class_weight

import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.utils import to_categorical

In [3]:
BASE_DIR = r"C:\Users\Saranya\OneDrive\Desktop\Papers\Ophthalmology\Dataset"

csv_path = os.path.join(BASE_DIR, "trainLabels.csv")

df = pd.read_csv(csv_path)

print("Preview of CSV:")
print(df.head())

images_folder = os.path.join(BASE_DIR, "resized_train")

df['file_path'] = df['image'].apply(lambda x: os.path.join(images_folder, f"{x}.jpeg"))

print("DataFrame with file_path:")
print(df.head())

Preview of CSV:
      image  level
0   10_left      0
1  10_right      0
2   13_left      0
3  13_right      0
4   15_left      1
DataFrame with file_path:
      image  level                                          file_path
0   10_left      0  C:\Users\Saranya\OneDrive\Desktop\Papers\Ophth...
1  10_right      0  C:\Users\Saranya\OneDrive\Desktop\Papers\Ophth...
2   13_left      0  C:\Users\Saranya\OneDrive\Desktop\Papers\Ophth...
3  13_right      0  C:\Users\Saranya\OneDrive\Desktop\Papers\Ophth...
4   15_left      1  C:\Users\Saranya\OneDrive\Desktop\Papers\Ophth...


In [4]:
print("Distribution of DR levels:")
print(df['level'].value_counts())

Distribution of DR levels:
level
0    25810
2     5292
1     2443
3      873
4      708
Name: count, dtype: int64


In [8]:
train_df, val_df = train_test_split(
    df,
    test_size=0.2,
    stratify=df['level'],
    random_state=42
)
y_train = train_df['level'].values

In [9]:
classes = np.unique(y_train) 

class_weights = compute_class_weight(
    class_weight='balanced',
    classes=classes,
    y=y_train
)

class_weights_dict = dict(enumerate(class_weights))
print("Class Weights:", class_weights_dict)

Class Weights: {0: 0.2721813250678032, 1: 2.8761514841351077, 2: 1.3273500236183278, 3: 8.051575931232092, 4: 9.929328621908127}


In [13]:
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=15,
    zoom_range=0.1,
    horizontal_flip=True
)

train_gen = train_datagen.flow_from_dataframe(
    dataframe=train_df,
    x_col='file_path',    
    y_col='level',        
    target_size=(224, 224),
    batch_size=16,
    class_mode='raw',     
    shuffle=True
)

val_datagen = ImageDataGenerator(rescale=1./255)

val_gen = val_datagen.flow_from_dataframe(
    dataframe=val_df,
    x_col='file_path',
    y_col='level',
    target_size=(224, 224),
    batch_size=16,
    class_mode='raw',
    shuffle=False
)

Found 28100 validated image filenames.
Found 7026 validated image filenames.


In [15]:
model = models.Sequential([
    layers.Conv2D(32, (3, 3), activation='relu', input_shape=(224, 224, 3)),
    layers.MaxPooling2D((2, 2)),

    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),

    layers.Conv2D(128, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),

    layers.Flatten(),
    layers.Dense(128, activation='relu'),
    layers.Dropout(0.3),
    layers.Dense(5, activation='softmax') 
])

model.compile(
    optimizer='adam',
    loss='sparse_categorical_crossentropy',  
    metrics=['accuracy']
)

model.summary()

model.fit(
    train_gen,
    validation_data=val_gen,
    epochs=10,
    class_weight=class_weights_dict 
)

Model: "sequential_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ conv2d_6 (Conv2D)                    │ (None, 222, 222, 32)        │             896 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_6 (MaxPooling2D)       │ (None, 111, 111, 32)        │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_7 (Conv2D)                    │ (None, 109, 109, 64)        │          18,496 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_7 (MaxPooling2D)       │ (None, 54, 54, 64)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_8 (Conv2D)                    │ (None, 52, 52, 128)         │          73,856 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_8 (MaxPooling2D)       │ (None, 26, 26, 128)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ flatten_2 (Flatten)                  │ (None, 86528)               │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_4 (Dense)                      │ (None, 128)                 │      11,075,712 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_2 (Dropout)                  │ (None, 128)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_5 (Dense)                      │ (None, 5)                   │             645 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 11,169,605 (42.61 MB)

 Trainable params: 11,169,605 (42.61 MB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/10
1757/1757 ━━━━━━━━━━━━━━━━━━━━ 1809s 1s/step - accuracy: 0.1040 - loss: 1.6233 - val_accuracy: 0.0249 - val_loss: 1.6072
Epoch 2/10
1757/1757 ━━━━━━━━━━━━━━━━━━━━ 1685s 958ms/step - accuracy: 0.0839 - loss: 1.6227 - val_accuracy: 0.0249 - val_loss: 1.5997
Epoch 3/10
1757/1757 ━━━━━━━━━━━━━━━━━━━━ 1650s 938ms/step - accuracy: 0.2176 - loss: 1.6136 - val_accuracy: 0.0202 - val_loss: 1.6160
Epoch 4/10
1757/1757 ━━━━━━━━━━━━━━━━━━━━ 1654s 940ms/step - accuracy: 0.0691 - loss: 1.6154 - val_accuracy: 0.1506 - val_loss: 1.6111
Epoch 5/10
1757/1757 ━━━━━━━━━━━━━━━━━━━━ 1537s 873ms/step - accuracy: 0.1800 - loss: 1.5773 - val_accuracy: 0.0249 - val_loss: 1.6133
Epoch 6/10
1757/1757 ━━━━━━━━━━━━━━━━━━━━ 1674s 952ms/step - accuracy: 0.0682 - loss: 1.6299 - val_accuracy: 0.1506 - val_loss: 1.6099
Epoch 7/10
1757/1757 ━━━━━━━━━━━━━━━━━━━━ 1411s 802ms/step - accuracy: 0.1167 - loss: 1.6284 - val_accuracy: 0.7347 - val_loss: 1.5977
Epoch 8/10
1757/1757 ━━━━━━━━━━━━━━━━━━━━ 1537s 873ms/step